In [1]:
import sys
import time
try:
    # For python2
    from itertools import izip as zip
    LARGE_NUMBER = sys.maxint
except ImportError:
    # For python3
    LARGE_NUMBER = sys.maxsize

import numpy as np


class Dataset(object):
    def __init__(self, X, y):
        self.X = X
        self.y = y


class TreeNode(object):
    def __init__(self):
        self.is_leaf = False
        self.left_child = None
        self.right_child = None
        self.split_feature_id = None
        self.split_val = None
        self.weight = None

    def _calc_split_gain(self, G, H, G_l, H_l, G_r, H_r, lambd):
        """
        Loss reduction
        (Refer to Eq7 of Reference[1])
        """
        def calc_term(g, h):
            return np.square(g) / (h + lambd)
        return calc_term(G_l, H_l) + calc_term(G_r, H_r) - calc_term(G, H)

    def _calc_leaf_weight(self, grad, hessian, lambd):
        """
        Calculate the optimal weight of this leaf node.
        (Refer to Eq5 of Reference[1])
        """
        return np.sum(grad) / (np.sum(hessian) + lambd)

    def build(self, instances, grad, hessian, shrinkage_rate, depth, param):
        """
        Exact Greedy Alogirithm for Split Finidng
        (Refer to Algorithm1 of Reference[1])
        """
        assert instances.shape[0] == len(grad) == len(hessian)
        if depth > param['max_depth']:
            self.is_leaf = True
            self.weight = self._calc_leaf_weight(grad, hessian, param['lambda']) * shrinkage_rate
            return
        G = np.sum(grad)
        H = np.sum(hessian)
        best_gain = 0.
        best_feature_id = None
        best_val = 0.
        best_left_instance_ids = None
        best_right_instance_ids = None
        for feature_id in range(instances.shape[1]):
            G_l, H_l = 0., 0.
            sorted_instance_ids = instances[:,feature_id].argsort()
            for j in range(sorted_instance_ids.shape[0]):
                G_l += grad[sorted_instance_ids[j]]
                H_l += hessian[sorted_instance_ids[j]]
                G_r = G - G_l
                H_r = H - H_l
                current_gain = self._calc_split_gain(G, H, G_l, H_l, G_r, H_r, param['lambda'])
                if current_gain > best_gain:
                    best_gain = current_gain
                    best_feature_id = feature_id
                    best_val = instances[sorted_instance_ids[j]][feature_id]
                    best_left_instance_ids = sorted_instance_ids[:j+1]
                    best_right_instance_ids = sorted_instance_ids[j+1:]
        if best_gain < param['min_split_gain']:
            self.is_leaf = True
            self.weight = self._calc_leaf_weight(grad, hessian, param['lambda']) * shrinkage_rate
        else:
            self.split_feature_id = best_feature_id
            self.split_val = best_val

            self.left_child = TreeNode()
            self.left_child.build(instances[best_left_instance_ids],
                                  grad[best_left_instance_ids],
                                  hessian[best_left_instance_ids],
                                  shrinkage_rate,
                                  depth+1, param)

            self.right_child = TreeNode()
            self.right_child.build(instances[best_right_instance_ids],
                                   grad[best_right_instance_ids],
                                   hessian[best_right_instance_ids],
                                   shrinkage_rate,
                                   depth+1, param)

    def predict(self, x):
        if self.is_leaf:
            return self.weight
        else:
            if x[self.split_feature_id] <= self.split_val:
                return self.left_child.predict(x)
            else:
                return self.right_child.predict(x)


class Tree(object):
    ''' Classification and regression tree for tree ensemble '''
    def __init__(self):
        self.root = None

    def build(self, instances, grad, hessian, shrinkage_rate, param):
        assert len(instances) == len(grad) == len(hessian)
        self.root = TreeNode()
        current_depth = 0
        self.root.build(instances, grad, hessian, shrinkage_rate, current_depth, param)

    def predict(self, x):
        return self.root.predict(x)


class GBT(object):
    def __init__(self):
        self.params = {'gamma': 0.,
                       'lambda': 1.,
                       'min_split_gain': 0.1,
                       'max_depth': 5,
                       'learning_rate': 0.3,
                       }
        self.best_iteration = None

    def _calc_training_data_scores(self, train_set, models):
        if len(models) == 0:
            return None
        X = train_set.X
        scores = np.zeros(len(X))
        for i in range(len(X)):
            scores[i] = self.predict(X[i], models=models)
        return scores
    

    def _calc_l2_gradient(self, train_set, scores):
        labels = train_set.y
        hessian = np.full(len(labels), 2)
        if scores is None:
            grad = np.random.uniform(size=len(labels))
        else:
            grad = np.array([2 * (labels[i] - scores[i]) for i in range(len(labels))])
        return grad, hessian

    def _calc_gradient(self, train_set, scores):
        """For now, only L2 loss is supported"""
        return self._calc_l2_gradient(train_set, scores)

    def _calc_l2_loss(self, models, data_set):
        errors = []
        for x, y in zip(data_set.X, data_set.y):
            errors.append(y - self.predict(x, models))
        return np.mean(np.square(errors))

    def _calc_loss(self, models, data_set):
        """For now, only L2 loss is supported"""
        return self._calc_l2_loss(models, data_set)

    def _build_learner(self, train_set, grad, hessian, shrinkage_rate):
        learner = Tree()
        learner.build(train_set.X, grad, hessian, shrinkage_rate, self.params)
        return learner

    def train(self, params, train_set, num_boost_round=20, valid_set=None, early_stopping_rounds=5):
        self.params.update(params)
        models = []
        shrinkage_rate = 1.
        best_iteration = None
        best_val_loss = LARGE_NUMBER
        train_start_time = time.time()

        print("Training until validation scores don't improve for {} rounds."
              .format(early_stopping_rounds))
        for iter_cnt in range(num_boost_round):
            iter_start_time = time.time()
            scores = self._calc_training_data_scores(train_set, models)
            grad, hessian = self._calc_gradient(train_set, scores)
            learner = self._build_learner(train_set, grad, hessian, shrinkage_rate)
            if iter_cnt > 0:
                shrinkage_rate *= self.params['learning_rate']
            models.append(learner)
            train_loss = self._calc_loss(models, train_set)
            val_loss = self._calc_loss(models, valid_set) if valid_set else None
            val_loss_str = '{:.10f}'.format(val_loss) if val_loss else '-'
            print("Iter {:>3}, Train's L2: {:.10f}, Valid's L2: {}, Elapsed: {:.2f} secs"
                  .format(iter_cnt, train_loss, val_loss_str, time.time() - iter_start_time))
            if val_loss is not None and val_loss < best_val_loss:
                best_val_loss = val_loss
                best_iteration = iter_cnt
            if iter_cnt - best_iteration >= early_stopping_rounds:
                print("Early stopping, best iteration is:")
                print("Iter {:>3}, Train's L2: {:.10f}".format(best_iteration, best_val_loss))
                break

        self.models = models
        self.best_iteration = best_iteration
        print("Training finished. Elapsed: {:.2f} secs".format(time.time() - train_start_time))

    def predict(self, x, models=None, num_iteration=None):
        if models is None:
            models = self.models
        assert models is not None
        return np.sum(m.predict(x) for m in models[:num_iteration])

In [5]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# from tinygbt import Dataset, GBT


print('Load data...')
data = pd.read_csv('facies_vectors.csv')
data = data.fillna(data['PE'].mean())
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
well = 'KIMZEY A'
test = data[data['Well Name'] == well]
train = data[data['Well Name'] != well]
X_train = train[feature_names].values 
y_train = train['Facies'].values 
X_test = test[feature_names].values 
y_test = test['Facies'].values 

train_data = Dataset(X_train, y_train)
eval_data = Dataset(X_test, y_test)

params = {}

print('Start training...')
gbt = GBT()
gbt.train(params,
          train_data,
          num_boost_round=20,
          valid_set=eval_data,
          early_stopping_rounds=5)

print('Start predicting...')
y_pred = []
for x in X_test:
    y_pred.append(gbt.predict(x, num_iteration=gbt.best_iteration))

def accuracy(y_pred, y):
    acc = 0
    for index in range (len(y)):
        if (y[index] == y_pred[index]):
            acc += 1
    return acc/len(y)

print("Accuracy ", accuracy(y_pred, y_test))
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Load data...
Start training...
Training until validation scores don't improve for 5 rounds.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:209: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


Iter   0, Train's L2: 23.9603734376, Valid's L2: 25.9775635996, Elapsed: 5.45 secs
Iter   1, Train's L2: 8.5676006769, Valid's L2: 6.8969110854, Elapsed: 5.61 secs
Iter   2, Train's L2: 7.6388028112, Valid's L2: 6.0243508594, Elapsed: 5.62 secs
Iter   3, Train's L2: 7.4007471123, Valid's L2: 5.8150365710, Elapsed: 5.70 secs
Iter   4, Train's L2: 7.3650547585, Valid's L2: 5.7738123750, Elapsed: 5.79 secs
Iter   5, Train's L2: 7.3455167012, Valid's L2: 5.7574070372, Elapsed: 5.78 secs
Iter   6, Train's L2: 7.3396126029, Valid's L2: 5.7524896939, Elapsed: 5.96 secs
Iter   7, Train's L2: 7.3386949618, Valid's L2: 5.7514166800, Elapsed: 5.94 secs
Iter   8, Train's L2: 7.3381652186, Valid's L2: 5.7509756305, Elapsed: 5.94 secs
Iter   9, Train's L2: 7.3380063321, Valid's L2: 5.7508433508, Elapsed: 5.95 secs
Iter  10, Train's L2: 7.3379613051, Valid's L2: 5.7508036701, Elapsed: 6.02 secs
Iter  11, Train's L2: 7.3379470066, Valid's L2: 5.7507917662, Elapsed: 6.05 secs
Iter  12, Train's L2: 7.33

In [6]:
y_pred


[2.4290819719912244,
 2.443196377982123,
 2.441830485363464,
 2.4374174870894976,
 2.435553421660977,
 1.9966533245892308,
 1.9966533245892308,
 1.5488140556307237,
 1.5488140556307237,
 1.5488140556307237,
 4.101472283992752,
 2.435553421660977,
 1.6317820460736892,
 2.724832465994971,
 2.7588532960675027,
 2.724832465994971,
 2.4609091429991308,
 2.4771014988950766,
 2.0382014018233314,
 2.0382014018233314,
 2.1269206257644826,
 2.1269206257644826,
 2.0031747418110317,
 2.0031747418110317,
 2.0031747418110317,
 2.0031747418110317,
 2.0031747418110317,
 2.0031747418110317,
 2.0031747418110317,
 2.1269206257644826,
 2.0023165063222788,
 2.1269206257644826,
 2.1269206257644826,
 2.5658207228362278,
 2.442074838882777,
 2.0031747418110317,
 2.0031747418110317,
 2.1914874528800894,
 2.1914874528800894,
 2.2127125233682134,
 2.217052735612049,
 2.463299909370901,
 1.652001286622045,
 1.801884710101609,
 3.0465589739992613,
 4.353996521454094,
 2.7823721615110815,
 2.6092588016266554,
 2.89